# Neural Network

In this R Jupyter notebook, we go through the code implementation for a neural network model. In particular, we are interested in whether the probability of getting $A/A-$ is related to both midterm 1 score and gender.

## 1. Loading the data

If the data size is not large, it may be convenient to make two data vectors manually as follows.

In [1]:
midterm1 <- c(23, 22.5, 21.5, 21.25, 13, 25, 14.5, 20, 18, 18, 19.5, 
              25, 23, 19, 24, 20, 24, 24, 18.5, 18, 16, 21.85, 25, 19.5) # explanatory variable 1

gender <- c(0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1) # explanatory variable 2

aam <- c(1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0) # response variable

If the data size is not small enough for a manual input, we have to load the data. The following is an example of loading the data "aam.csv" from my desktop folder. Please refer to the previous jupyter notebooks, especially "SLR.ipynb", which contains details of loading the data.

In [2]:
setwd("~/Desktop/")
data <- read.csv("aam.csv", header = TRUE) # R is case-sensitive
head(data)
aam <- data[, 1]
midterm1 <- data[, 2]
gender <- data[, 3]

,AAm,midterm1,gender
,<int>,<dbl>,<int>
1,1,23.00,0
2,1,22.50,0
3,1,21.50,0
4,0,21.25,0
5,0,13.00,0
6,1,25.00,0


For Window users, the fist line of the code above is replaced with

setwd("C:/Users/hyungsuktak/Desktop/")

If the data file is a .txt file, then the following code will be used instead.

midterm <- read.table("aam.txt", header = TRUE)

To check whether the data are correctly loaded, we use the function "head" that shows the first six lines of the data. To avoid any mistake, it is better to check whether the first row is correctly loaded. In this case, the first row correctly starts with (1, 23, 0).

Next, we designate each column of the data to an object, as we input manually in the beginning. 

Now the data are loaded, and we are ready to conduct a multiple linear regression analysis.

## 2. Fitting a neural network model

We are going to fit and compare a neural network model that has two hidden layers with three and two nodes, respectively.  

The Sigmoid activation function will be used at each node. As shown during the class, such a neural network model for a binary classification with the Sigmoid activation function (at least in the last layer) can be considered as a logistic regression model with a complicated link function. In this sense, the response variables ($Y_i$'s) are  independent Bernoulli random variables  with probabilities of getting $A/A-$ equal to $\theta_i=h_w(x_i)$'s, i.e.,

$$
Y_i\stackrel{\textrm{ind.}}{\sim} \textrm{Bern}(h_w(x_i)),
$$

where $x_i=(1, x_{i1}, x_{i2})^{\top}$ and $h_w(x_i)$ is the activated value in the last layer for the $i$-th person, ranging between 0 and 1 due to the Sigmoid activation function. This function $h_w(x_i)$ represents the entire connections throughout the neural net with an input $x_i$, so this is a very complicated (but still deterministic) function. The log-likelihood function of this model is

$$
\ell(w)= \sum_{i=1}^n\left[y_i\log(h_w(x_i)) + (1-y_i)\log(1-h_w(x_i))\right].
$$


We estimate weights $w$ by the values that maximize the log-likelihood function above, or equivalently the values that minimize the negative log-likelihood function, called a cost function in machine learning.

In R, there are many packages to fit neural network models, but here we use "neuralnet". To use this package, we need to intall and load it on R as follows.

In [3]:
install.packages("neuralnet")
library(neuralnet)


The downloaded binary packages are in
	/var/folders/yk/fk6w4crd17v4g_qflhhv1xlc0000gn/T//RtmpAxhPxi/downloaded_packages


To measure prediction accuracy, we use a cross-validation by splitting the data set into training and test sets. We randomly select 15 people and assign them to the training set and the rest of people to the test set.

In [4]:
data.all <- data.frame(midterm1 = midterm1, gender = gender, aam = aam)
random.index <- sample(1 : 24, 15, replace = FALSE)
random.index
data.train <- data.all[random.index, ]
data.test <- data.all[-random.index, ]

[1]  9 16 20 21 24  2 10  3 22  8 19  6  4 14 15

The function "sample" above draws 15 random numbers from integers between 1 and 24 without sampling any number twice (replace = FALSE). These 15 random numbers are saved in "random.index". Note that "data.all[random.index, ]" makes a sub-dataset of the observations corresponding to "random.index", and "data.all[-random.index, ]" makes a sub-dataset of the observations NOT corresponding to "random.index" (i.e., a complement set).

Then, the code below fits the  neural network model and saves the fits in the object "res".

In [5]:
res <- neuralnet(aam ~ midterm1 + gender, data = data.train, 
                hidden = c(3, 2), act.fct = "logistic", linear.output = FALSE)

The argument "hidden = c(3, 2)" means that there are two layers, the first of which has three nodes and the second has two nodes. If "linear.output = TRUE", then the identity function is used regardless of the specified activation function ("act.fct = logistic").

We can simply visualize the fits using the "plot" function and display all of the details using the function "print" below. For some reason, the plot does not work in Jupyter notebook, although this code will work on your local R console. 

In [6]:
plot(res)
print(res)

$call
neuralnet(formula = aam ~ midterm1 + gender, data = data.train, 
    hidden = c(3, 2), act.fct = "logistic", linear.output = FALSE)

$response
   aam
9    0
16   1
20   0
21   0
24   0
2    1
10   0
3    1
22   1
8    1
19   0
6    1
4    0
14   0
15   1

$covariate
   midterm1 gender
9     18.00      0
16    20.00      0
20    18.00      0
21    16.00      0
24    19.50      1
2     22.50      0
10    18.00      0
3     21.50      0
22    21.85      1
8     20.00      1
19    18.50      1
6     25.00      0
4     21.25      0
14    19.00      0
15    24.00      1

$model.list
$model.list$response
[1] "aam"

$model.list$variables
[1] "midterm1" "gender"  


$err.fct
function (x, y) 
{
    1/2 * (y - x)^2
}
<bytecode: 0x7f83726a9858>
<environment: 0x7f83726adad0>
attr(,"type")
[1] "sse"

$act.fct
function (x) 
{
    1/(1 + exp(-x))
}
<bytecode: 0x7f83726d3e08>
<environment: 0x7f83726c4d78>
attr(,"type")
[1] "logistic"

$linear.output
[1] FALSE

$data
   midterm1 gender aam
9     1

## 3. Making predictions

Finally, let's make predictions using the test set and calculate the mean squared error.

In [7]:
pred.temp <- compute(res, data.test)
est.prob <- pred.temp$net.result
pred <- ifelse(est.prob > 0.5, 1, 0)
mse <- mean((data.test$aam - pred)^2)
round(mse, 3)

[1] 0.222